<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
سفر نمی‌روم دگر!
</font>
</h1>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    احتمالاً هنگامی‌که بخواهید برای تعطیلات نوروز به یک شهر مانند ارومیه سفر کنید، از طریق موبایل و یا لپ‌تاپ‌ و از مسیر جست‌وجوی گوگل، پیامک تبلیغاتی و یا به‌صورت مستقیم، وارد یک سایت رزرو هتل می‌شوید و بعد از ساخت حساب کاربری، اقدام به جست‌وجو کرده و مقصد و تاریخ ورود/خروج خود را وارد می‌کنید. در نتیجه لیستی از هتل‌ها برای شما، نشان داده‌ می‌شود و در نهایت، شما با کلیک بر روی هتل‌های مختلف، اقدام به مقایسه ی آن‌ها کرده و شاید (🙃) در نهایت یکی از آن‌ها را رزرو کنید.
<br>
در این پروژه شما پس از پیش‌پردازش داده‌ها و حل چالش‌های مربوط به آن، مدلی را آموزش می‌دهید که بر اساس اطلاعات جست‌وجوی کاربران و سایر ویژگی‌های مرتبط با آن، بتواند پیش‌بینی کند که آیا یک کاربر، هتل مشاهده‌شده را رزرو خواهد کرد یا خیر. بدین ترتیب، می‌توان در لحظه برای هر کاربر، متناسب با پیش‌بینی رزرو، تصمیم متناسبی همانند ارائه‌ی تخفیف و یا پیشنهاد سایر هتل‌ها اتخاذ کرد.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیاز را وارد کنید. در صورتی‌که نیاز به کتابخانه‌های دیگری دارید نیز می‌توانید آن‌ها را فراخوانی کنید.
</font>
</p>

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.feature_selection import VarianceThreshold
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import category_encoders as ce
from imblearn.over_sampling import SMOTE
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [4]:
import keras
print(keras.__version__)

3.9.2


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>



<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به‌دلیل حجم بالای مجموعه‌داده نیاز است آن را جداگانه از <a href="https://drive.google.com/file/d/1jIl8uXsrdRz3ZBxljjQNo0ssDT8LbJBx/view?usp=sharing" target="_blank">این لینک</a> دانلود کرده و فایل‌های داخل آن را استخراج کنید.  توجه داشته باشید که اندازه‌ی مجموعه‌داده حدود ۱۸۰ مگابایت است و اگر قصد دانلود این حجم از داده را ندارید پیشنهاد می‌کنیم از محیط گوگل کولب استفاده کرده و به‌صورت مستقیم با دستور زیر این فایل را روی کولب خود بارگیری کنید.
</font>
</p>

`!gdown 1jIl8uXsrdRz3ZBxljjQNo0ssDT8LbJBx`

<details dir="rtl" style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<summary dir="rtl" style="color:orange"><b>رفع مشکل عدم امکان دانلود فایل با <code>gdown</code></b></summary>

اگر در هنگام دانلود فایل به‌کمک <code>gdown</code> با خطای عدم اجازه‌ی دسترسی مواجه شدید، کافیست کد زیر را اجرا کرده و دوباره امتحان کنید.

<span dir=ltr style="direction:ltr;">

`!pip install --upgrade --no-cache-dir gdown`

</span>
</details>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این سوال، شما به داده‌های جست‌وجوی کاربران دسترسی دارید. فایل آموزش (<code>train.csv</code>)، نتایج جست‌وجوی کاربران را در یک بازه‌ی حدوداً دو ساله شامل می‌شود و حدود ۷ میلیون سطر دارد. فایل آزمون (<code>test.csv</code>) دارای ۳۰۰ هزار سطر از اطلاعات جست‌وجوی کاربران در بازه‌ی حدود ۱ ماه پس از آخرین جست‌وجوی موجود در فایل آموزش است (این سطرها، به‌صورت تصادفی از بین چند میلیون سطر آن ماه، انتخاب شده‌اند).
جدول زیر، ستون‌های موجود در فایل آموزش را توضیح می‌دهد. توجه داشته‌ باشید که هر ردیف این مجموعه‌داده، نشان‌دهنده‌ی جست‌وجوی یک کاربر جهت رزرو هتل است.
     همچنین در مجموعه‌داده‌ی آزمون، ستون <code>is_booking</code> وجود ندارد، زیرا در ادامه از این ستون برای ساخت متغیر هدف مسئله استفاده خواهیم کرد.
    <br> نحوه‌ی روبه‌رو شدن شما با مجموعه‌داده‌ای با این حجم، جزو یکی از چالش‌ها و اهداف طراحی این تمرین بوده‌است.

</font>
</p>


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

| نام ستون | توضیحات ستون |
|:----------:|:------------------:|
| <code>user</code> | شناسه کاربر |
| <code>user_location_country</code> | شناسه‌ی کشور کاربر |
| <code>user_location_region</code> | شناسه‌ی منطقه‌ی کاربر |
| <code>user_location_city</code> | شناسه‌ی شهر کاربر |
| <code>destination_distance</code> | فاصله‌ی فیزیکی بین کاربر و هتل در زمان جست‌وجو‌ (مقدار <code>null</code> به معنی عدم توانایی محاسبه فاصله است) |
| <code>search_date</code> | زمان انجام جست‌وجو |
| <code>is_mobile</code> | آیا کاربر با دستگاه موبایل جست‌وجو انجام داده است؟ |
| <code>is_package</code> | آیا کاربر در حال جست‌وجوی هتل به‌همراه بلیط حمل‌و‌نقل (اتوبوس، هواپیما یا قطار) است؟|
| <code>channel</code> | کاربر از چه کانالی وارد سایت شده‌است؟ (تبلیغات پیامکی، تبلیغات شبکه‌ی اجتماعی، ورود مستقیم و ...)  |
| <code>Search_count</code> |تعداد جست‌وجو‌های مشابه در لحظه‌ی کاربر (در همان نشست یا ‌<i>session</i>)|
| <code>checkIn_date</code> | تاریخ ورود به هتل |
| <code>checkOut_date</code> | تاریخ خروج از هتل |
| <code>n_adults</code> | تعداد افراد بالغ اعلام‌شده جهت رزرو هتل |
| <code>n_children</code> | تعداد کودکان اعلام‌شده جهت رزرو هتل |
| <code>n_rooms</code> | تعداد اتاق مورد نظر برای رزرو |
| <code>destination</code> | شناسه‌ی محل هتل مورد نظر کاربر (با هم‌پوشانی همچون نیویورک، نیویورک سیتی، سنترال پارک و غیره) |
| <code>destination_type</code> | کد نوع هتل مورد نظر کار‌بر |
| <code>hotel_continent</code> | شناسه‌ی قاره‌ی هتل مورد نظر کاربر |
| <code>hotel_country</code> | شناسه‌ی کشور هتل مورد نظر کاربر |
| <code>hotel_market</code> | ناحیه‌ای که هتل مورد نظر کاربر در آن قرار گرفته‌است (بدون هم‌پوشانی همچون نیویورک، بوستون و غیره)|
| <code>hotel_category</code> | گرو‌ه‌بندی هتلی که جزییاتش را مشاهده می‌کنند. این گروه‌بندی می‌تواند بر اساس مواردی مانند چندستاره بودن و یا نوع هتل باشد. |
| <code>is_booking<code> | آیا کاربر در نهایت، هتل مشاهده‌شده را رزرو کرده یا خیر؟ |


</font>
</div>
</center>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خوانش مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
   در ابتدا نیاز است فایل‌ مجموعه‌داده را بخوانید. داده‌‌های آموزش را می‌توانید از فایل <code>train.csv</code> موجود در پوشه‌ی <code>data</code> بخوانید و از نمونه‌های موجود در آن برای آموزش مدل استفاده کنید.
 عملکرد مدل شما روی داده‌ی <code>test.csv</code> برسی می‌شود.
</font>
</p>

In [5]:
train = pd .read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

train.head()

user  user_location_country  user_location_region  user_location_city  \
0   461899                      3                    50                5703   
1    13796                     66                   174               21177   
2  1128575                    205                   155               14703   
3  1080476                     69                   761               41949   
4  1080476                     69                   761               41949   

   destination_distance          search_date  is_mobile  is_package  channel  \
0                   NaN  2013-01-07 00:00:02          0           0        9   
1             5713.6206  2013-01-07 00:00:06          0           0        9   
2              795.7298  2013-01-07 00:00:06          0           0        9   
3                   NaN  2013-01-07 00:00:17          0           1        9   
4                   NaN  2013-01-07 00:00:23          0           1        9   

   search_count  ... n_adults n_children  n_rooms  destination  \
0             1  ...        2          1        1          669   
1             3  ...        1          0        1         8821   
2             1  ...        1          0        1        25064   
3             1  ...        2          0        1         7635   
4             1  ...        2          0        1         7635   

   destination_type  hotel_continent  hotel_country  hotel_market  \
0                 3                2             50           212   
1                 1                6             17            30   
2                 6                2             50          1230   
3                 3                2             50           675   
4                 3                2             50           675   

   hotel_category  is_booking  
0              41           0  
1              58           0  
2              91           0  
3              10           0  
4              10           0  

[5 rows x 22 columns]

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
اکتشاف داده‌ها و مهندسی ویژکی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در مرحله اول به بررسی و اکتشاف داده‌ها و مهندسی ویژگی می‌پردازیم. در ابتدا با حذف ستون  <code>user</code> سعی در کم‌کردن حجم داده‌ها می‌کنیم، این عمل با توجه به یکتایی این مقادیر تاثیری در عملکرد مدل نخواهد داشت.
</font>
</p>

In [6]:
train=train.drop(["user"],axis=1)
test=test.drop(["user"],axis=1)

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
حذف مقادیر گم‌شده
</font>
</h3>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
   ابتدا با بررسی مقادیر گم‌شده در ستون‌های <code>checkIn_date</code> و <code>checkOut_date</code> این مقادیر را مدیریت کنید. در صورتی‌که حداقل ۹۹ درصد نمونه‌های شامل مقادیر گم‌شده در هر ستون دارای <b style="color:#0099cc">متغیر هدف</b> با مقدار <code>0</code> باشند، آن‌ها را حذف کنید، در غیر این‌صورت از روشی دیگری جهت مدیریت آن‌ها بهره ببرید.
</font>
</p>

In [7]:
train = train.dropna(subset=["checkOut_date"])

train = train.dropna(subset=["checkIn_date"])


In [8]:
mean_distances = train.groupby(['user_location_city', 'destination'])['destination_distance'].mean().reset_index()
mean_dict = mean_distances.set_index(['user_location_city', 'destination'])['destination_distance'].to_dict()
def fill_missing_distance(row):
    if pd.isna(row['destination_distance']):
        # گرفتن میانگین برای ترکیب user_location_city و destination
        key = (row['user_location_city'], row['destination'])
        return mean_dict.get(key, np.nan)  # اگر میانگین نبود، NaN برگردون
    return row['destination_distance']
train['destination_distance'] = train.apply(fill_missing_distance, axis=1)
train['destination_distance']=train['destination_distance'].fillna(0)

In [9]:
mean_distances_test = test.groupby(['user_location_city', 'destination'])['destination_distance'].mean().reset_index()
mean_dict_test = mean_distances_test.set_index(['user_location_city', 'destination'])['destination_distance'].to_dict()

def fill_missing_distance_test(row):
    if pd.isna(row['destination_distance']):
        # گرفتن میانگین برای ترکیب user_location_city و destination
        key = (row['user_location_city'], row['destination'])
        return mean_dict_test.get(key, np.nan)  # اگر میانگین نبود، NaN برگردون
    return row['destination_distance']

print(test["destination_distance"].isna().sum())
test['destination_distance'] = test.apply(fill_missing_distance_test, axis=1)
test['destination_distance']=test['destination_distance'].fillna(0)
print(test["destination_distance"].isna().sum())


98670
0


In [10]:
# TODO : change type of columns in time_columns into datetime64[ns]

time_columns = ['search_date', 'checkIn_date', 'checkOut_date']
for timecol in time_columns:
    train[timecol]=pd.to_datetime(train[timecol])
    test[timecol]=pd.to_datetime(test[timecol])

In [11]:
# TODO : Add Days of Stay 

train['duration'] = (train["checkOut_date"]-train["checkIn_date"]).dt.days
test['duration'] = (test["checkOut_date"]-test["checkIn_date"]).dt.days

# TODO : Add Days between search_date and checkIn_date

train['days_between'] = (train["checkIn_date"]-train["search_date"]).dt.days
test['days_between'] = (test["checkIn_date"]-test["search_date"]).dt.days

In [12]:
train['search_date_hour'] = train["search_date"].dt.hour
train['search_date_dayofweek'] = train["search_date"].dt.day_of_week
train['checkIn_date_dayofweek'] = train["checkIn_date"].dt.day_of_week
train['search_date_month'] = train["search_date"].dt.month
train['checkIn_date_month'] = train["checkIn_date"].dt.month

test['search_date_hour'] = test["search_date"].dt.hour
test['search_date_dayofweek'] = test["search_date"].dt.day_of_week
test['checkIn_date_dayofweek'] = test["checkIn_date"].dt.day_of_week
test['search_date_month'] = test["search_date"].dt.month
test['checkIn_date_month'] = test["checkIn_date"].dt.month

In [13]:
is_booked = train[train["is_booking"]==1]
not_booked = train[train["is_booking"]==0]


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
همان‌طور که در توضیحات ستون‌های مسئله دیدیم ستون <code>destination_distance</code> فاصله‌ی فیزیکی بین کاربر و هتل در زمان جست‌وجو‌ را نشان می‌دهد. از آنجایی که می‌توانیم بگوییم فاصله‌ی محاسبه‌شده دو شهر با یکدیگر همواره ثابت یا در یک محدوده است، برای پر کردن مقادیر گم‌شده‌ای که این فاصله را نشان می‌دهد، می‌توانیم از <code>user_location_city</code> و <code>destination</code> که بیانگر شناسه‌ی شهر کاربر و هتل جست‌وجو شده هستند، استفاده کنیم به‌طوری‌که نمونه‌‌های که ستون‌ <code>destination_distance</code> در آن گم‌شده است، با را با میانگین مقدار  <code>destination_distance</code> در ستون‌هایی که <code>user_location_city</code> و <code>destination</code> مشابه دارند پر کنیم.
</font>
</p>



<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
حال برای سایر موارد گم‌شده از عددی مانند <code>0</code> که در این ستون استفاده نشده است، استفاده می‌کنیم.
</font>
</p>


In [14]:
train.isna().sum()

user_location_country     0
user_location_region      0
user_location_city        0
destination_distance      0
search_date               0
is_mobile                 0
is_package                0
channel                   0
search_count              0
checkIn_date              0
checkOut_date             0
n_adults                  0
n_children                0
n_rooms                   0
destination               0
destination_type          0
hotel_continent           0
hotel_country             0
hotel_market              0
hotel_category            0
is_booking                0
duration                  0
days_between              0
search_date_hour          0
search_date_dayofweek     0
checkIn_date_dayofweek    0
search_date_month         0
checkIn_date_month        0
dtype: int64

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ویژگی‌های زمانی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
با بررسی ستون‌های مجموعه‌داده مشاهده می‌کنیم که سه ستون <code>checkOut_date</code>، <code>checkOut_date</code> و <code>search_date</code> بیانگر زمان هستند و می‌توان با تبدیل این مقادیر به <code>datetime64[ns]</code>  از آن‌ها اطلاعات و حتی ویژگی‌های جدید و مناسبی استخراج کرد. برای این‌کار از دستور زیر استفاده کنید. البته اکیداً توصیه می‌شود مستندات مربوط به کار با این نوع داده‌ها را بررسی کنید.
</font>
    
```python
df[‘time_type’] = pd.to_datetime(df[‘time_string’])
```
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در ابتدا شاید این سه ستون زمانی اطلاعات خوبی به مدل ندهند اما با بررسی دقیق‌تر می‌توانیم دو ویژگی جدید که تعیین‌کننده‌ی  <b>تعداد روز‌های بین زمان جست‌وجو و زمان تعیین‌شده برای ورود</b> و همچنین <b>تعداد روز‌های اقامت تعیین‌شده در جست‌وجو</b> هستند را به مجموعه داده اضافه کنیم. 
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همچنین می‌توانیم با بررسی ساعت، شماره روز‌ هفته و ماه زمان جست‌وجو و ورود به هتل، ارتباط آن‌ها را با رزرو هتل بررسی کنیم. بنابراین ستونی‌های جدول زیر را به مجموعه‌داده اضافه کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

| نام ستون | توضیحات ستون |
|:-:|:-:|
|<code>search_date_hour</code>|ساعت جست‌وجو |
|<code>search_date_dayofweek</code>|روز جست‌وجو در هفته|
|<code>checkIn_date_dayofweek</code>|روز ورود در هفته|
|<code>search_date_month</code>|ماه جست‌وجو|
|<code>checkIn_date_month</code>|ماه ورود|

</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
 حال برای ‌بررسی رابطه‌ی بین ویژگی‌هایی که در مرحله قبل به‌دست آوردیم با متغیر هدف (این‌که جست‌وجو به رزرو هتل منجر شده‌است یا خیر) دو دیتافرم <code>is_booked</code> و <code>not_booked</code> را براساس مجموعه‌داده‌ی آموزش ایجاد کنید، به‌طوری‌که دیتافرم اول فقط نمونه‌هایی که به رزرو ختم شده‌اند و دیتافریم دوم فقط نمونه‌هایی که به رزرو ختم نشده‌اند را شامل شود.
</font>
</p>


<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مصورسازی ۱
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
اکنون بیاید با استفاده از مصورسازی، بینش‌های بیشتری را از دل این داده‌ها استخراج کنیم.
در این مرحله نیاز است یک نمودار میله‌ای رسم کنید تا رابطه‌ی بین درصد جست‌وجو‌هایی که به رزرو هتل ختم شده‌اند و نشده‌اند‌ (هر کدام نسبت به دسته‌ی خودشان) در ساعات مختلف را نشان دهد. 
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
خروجی نمودار شما باید شبیه به تصویر زیر باشد.
</font>
</p>

<center>
<img src = 'https://quera.org/qbox/view/6TWauQwG58/search_hour.png'>
</center>

In [22]:
import plotly.graph_objects as go
# شمارش تعداد جستجوها برای هر ساعت
booked_counts = is_booked['search_date_hour'].value_counts().sort_index()
not_booked_counts = not_booked['search_date_hour'].value_counts().sort_index()

# کامل کردن ساعت‌ها (0 تا 23)
hours = list(range(24))
booked_counts = booked_counts.reindex(hours, fill_value=0)
not_booked_counts = not_booked_counts.reindex(hours, fill_value=0)

# محاسبه درصد نسبت به کل هر دسته
total_booked = booked_counts.sum()
total_not_booked = not_booked_counts.sum()
booked_percent = (booked_counts / total_booked ) if total_booked > 0 else pd.Series(0, index=hours)
not_booked_percent = (not_booked_counts / total_not_booked ) if total_not_booked > 0 else pd.Series(0, index=hours)

# تعریف traces
trace_is_booked = go.Bar(x=hours, y=booked_percent, name='Booked')
trace_not_booked = go.Bar(x=hours, y=not_booked_percent, name='Not Booked')

# *** از اینجا به بعد تغییر نمی‌کند ***
data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Search Hour', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./search_hour.json')


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این مرحله می‌توانید با توجه به زمان‌های مختلفی که جست‌وجو در آن انجام شده و رابطه‌ی بین آن با رزرو هتل، مقادیر این ستون را به دسته‌های کوچک‌تری افراز کنید (binning).
          به‌طور مثال به نمونه‌هایی که در آن‌ها جست‌وجو بین ساعت ۵ تا ۱۵ انجام شده‌است، مقدار <code>1</code>
          </font> دهید و به بقیه نمونه‌ها مقدار  <code>0</code> دهید. این عمل یک نوع مهندسی ویژگی به شمار می‌آید. توجه کنید این دسته‌بندی اختیاری بوده است و به‌طور مستقیم داوری نمی‌شود، اما ممکن است در عملکرد مدل شما تاثیر (مثبت یا منفی!) بگذارد، همچنین برای سایر ستون‌ها و ویژگی‌ها نیز با خلاقیت خودتان می‌توانید ویژگی‌های جدیدی خلق کنید.
</p>



In [19]:
train.head()

user_location_country  user_location_region  user_location_city  \
0                      3                    50                5703   
1                     66                   174               21177   
2                    205                   155               14703   
3                     69                   761               41949   
4                     69                   761               41949   

   destination_distance         search_date  is_mobile  is_package  channel  \
0              0.000000 2013-01-07 00:00:02          0           0        9   
1           5713.620600 2013-01-07 00:00:06          0           0        9   
2            795.729800 2013-01-07 00:00:06          0           0        9   
3            402.699108 2013-01-07 00:00:17          0           1        9   
4            402.699108 2013-01-07 00:00:23          0           1        9   

   search_count checkIn_date  ... hotel_category  is_booking  duration  \
0             1   2013-03-14  ...             41           0         1   
1             3   2013-01-19  ...             58           0         7   
2             1   2013-01-19  ...             91           0         3   
3             1   2013-05-29  ...             10           0         7   
4             1   2013-05-29  ...             10           0         7   

   days_between  search_date_hour  search_date_dayofweek  \
0            65                 0                      0   
1            11                 0                      0   
2            11                 0                      0   
3           141                 0                      0   
4           141                 0                      0   

   checkIn_date_dayofweek  search_date_month  checkIn_date_month  \
0                       3                  1                   3   
1                       5                  1                   1   
2                       5                  1                   1   
3                       2                  1                   5   
4                       2                  1                   5   

   search_hour_bin  
0                0  
1                0  
2                0  
3                0  
4                0  

[5 rows x 29 columns]

In [14]:
train['search_hour_bin'] = train['search_date_hour'].apply(lambda x: 1 if 5 <= x <= 15 else 0)
test['search_hour_bin'] = test['search_date_hour'].apply(lambda x: 1 if 5 <= x <= 15 else 0)

In [15]:
n_adults_mapping = {0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 3, 6: 3,7: 3 ,8: 3, 9: 3}
train['n_adults_mapped'] = train['n_adults'].map(n_adults_mapping).fillna(3)
test['n_adults_mapped'] = test['n_adults'].map(n_adults_mapping).fillna(3)

In [16]:
n_child_mapping = {0: 0, 1: 1, 2: 2, 3: 2, 4: 2, 5: 3, 6: 3,7: 3 ,8: 3, 9: 3}
train['n_child_mapped'] = train["n_children"].map(n_child_mapping).fillna(3)
test['n_child_mapped'] = test["n_children"].map(n_child_mapping).fillna(3)

In [17]:
train['total_persons'] = train['n_adults'] + train['n_children']
test['total_persons'] = test['n_adults'] + test['n_children']

In [18]:
train['persons_per_room'] = train['total_persons'] / (train['n_rooms'] + 1e-6)  # جلوگیری از تقسیم بر صفر
test['persons_per_room'] = test['total_persons'] / (test['n_rooms'] + 1e-6)
train['persons_per_room'] = train['persons_per_room'].clip(upper=10)  # محدود کردن مقادیر بزرگ
test['persons_per_room'] = test['persons_per_room'].clip(upper=10)

In [19]:
train['is_room_sufficient'] = (train['total_persons'] <= 4 * train['n_rooms']).astype(int)  # 1 اگر کافی، 0 اگر ناکافی
test['is_room_sufficient'] = (test['total_persons'] <= 4 * test['n_rooms']).astype(int)

In [20]:
def group_type(row):
    adults, children, rooms = row['n_adults'], row['n_children'], row['n_rooms']
    total = adults + children
    if adults == 1 and children == 0 and rooms == 1:
        return 0  # انفرادی
    elif adults == 2 and children == 0 and rooms == 1:
        return 1  # زوج
    elif children > 0:
        return 2  # خانواده
    elif total > 4 or rooms > 1:
        return 3  # گروه بزرگ
    else:
        return 4  # سایر
train['group_type'] = train.apply(group_type, axis=1)
test['group_type'] = test.apply(group_type, axis=1)

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مصورسازی ۲
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همانند نمودار قبل برای روز ورود به هتل  نیز نمودار را رسم کنید. دقت داشته باشید روز‌های هفته به‌صورت، دوشنبه، سه‌شنبه تا شنبه و به همین ترتیب باشد. به رعایت نیم‌فاصله در اسامی روزها توجه داشته باشید. کلمه‌ی سه‌شنبه و پنج‌شنبه دارای نیم‌فاصله است.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
خروجی نمودار شما باید به‌صورت زیر باشد.
</font>
</p>

<center>
<img src = 'https://quera.org/qbox/view/jqISJX07yD/checkIn_day.png'>
</center>


In [39]:
train["checkIn_date_month"].value_counts()

checkIn_date_month
8     839099
7     759274
12    744201
11    717246
10    714350
6     662528
9     646982
5     644047
4     571845
3     568868
2     411489
1     328779
Name: count, dtype: int64

In [21]:
import plotly.graph_objects as go
# شمارش تعداد جستجوها برای هر ساعت
booked_counts = is_booked["checkIn_date_dayofweek"].value_counts().sort_index()
not_booked_counts = not_booked["checkIn_date_dayofweek"].value_counts().sort_index()

# دیکشنری نگاشت روزهای هفته
day_mapping = {
    0: 'دوشنبه',
    1: 'سه\u200cشنبه',  # استفاده از نیم‌فاصله
    2: 'چهارشنبه',
    3: 'پنج\u200cشنبه',
    4: 'جمعه',
    5: 'شنبه',
    6: 'یکشنبه'
}
day_names = ['دوشنبه', 'سه\u200cشنبه', 'چهارشنبه', 'پنج\u200cشنبه', 'جمعه', 'شنبه', 'یکشنبه']
# تغییر ایندکس‌ها به نام‌های فارسی
booked_counts.index = booked_counts.index.map(day_mapping)
not_booked_counts.index = not_booked_counts.index.map(day_mapping)

# محاسبه درصد نسبت به کل هر دسته
total_booked = booked_counts.sum()
total_not_booked = not_booked_counts.sum()
booked_percent = (booked_counts / total_booked ) if total_booked > 0 else pd.Series(0, index=hours)
not_booked_percent = (not_booked_counts / total_not_booked ) if total_not_booked > 0 else pd.Series(0, index=hours)


# تعریف traces
trace_is_booked = go.Bar(x=day_names, y=booked_percent, name='Booked')
trace_not_booked = go.Bar(x=day_names, y=not_booked_percent, name='Not Booked')

# *** از اینجا به بعد تغییر نمی‌کند ***

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Day of Week', tickangle=45, automargin=True,
               tickvals = [0,1,2,3,4,5,6], ticktext= day_names
 ),
    yaxis=dict(title='Frequency'),
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./checkIn_day.json')

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مصورسازی ۳
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همانند نمودار قبل برای ماه ورود نیز نمودار را رسم کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
خروجی نمودار شما باید مشابه با تصویر زیر باشد.
</font>
</p>

<center>
<img src = 'https://quera.org/qbox/view/vdWwvplc3K/checkIn_date_month.png'>
</center>

In [23]:
booked_counts = is_booked["checkIn_date_month"].value_counts().sort_index()
not_booked_counts = not_booked["checkIn_date_month"].value_counts().sort_index()

ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
booked_counts.index = ticktext
not_booked_counts.index = ticktext

# محاسبه درصد نسبت به کل هر دسته
total_booked = booked_counts.sum()
total_not_booked = not_booked_counts.sum()
booked_percent = (booked_counts / total_booked ) if total_booked > 0 else pd.Series(0, index=hours)
not_booked_percent = (not_booked_counts / total_not_booked ) if total_not_booked > 0 else pd.Series(0, index=hours)

trace_not_booked = go.Bar(x = ticktext ,y=not_booked_percent  , name='Not Booked') 
trace_is_booked = go.Bar(x = ticktext , y=booked_percent, name='Booked') 

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]


layout = go.Layout(
    xaxis=dict(title='Month', tickangle=45, automargin=True,
             ticktext = ticktext ,tickvals = np.arange(0,12)),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./checkIn_date_month.json')


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در صورت علاقه می‌توانید ماه‌های سال را به فصل نیز تبدیل کنید (مانند پیشنهادی که برای ساعت جست‌وجو داده شد).
</p>


In [ ]:
# TODO (Optional): Make season column for checkIn_date_month or search_date_month

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مصورسازی ۴
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
نمودار خطی برای فاصله‌ی زمانی بین جست‌وجو و ورود را نیز رسم کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
خروجی نمودار شما باید به صورت زیر باشد.
</font>
</p>
<center>
<img src = 'https://quera.org/qbox/view/mEwuo0YTeE/days_between.png'>
</center>

In [24]:
booked_counts = is_booked["days_between"].value_counts().sort_index()
not_booked_counts = not_booked["days_between"].value_counts().sort_index()

total_booked = booked_counts.sum()
total_not_booked = not_booked_counts.sum()
booked_percent = (booked_counts / total_booked ) if total_booked > 0 else pd.Series(0, index=hours)
not_booked_percent = (not_booked_counts / total_not_booked ) if total_not_booked > 0 else pd.Series(0, index=hours)

trace_not_booked = go.Scatter(y = not_booked_percent , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y =  booked_percent, name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Days between search and checking time', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./days_between.json')

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مصورسازی ۵
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همچون نمودار بالا یک نمودار برای مدت اقامت نیز رسم کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
خروجی نمودار شما باید به‌صورت زیر باشد.
</font>
</p>

<center>
<img src = 'https://quera.org/qbox/view/iOh4Yzgkf6/los.png'>
</center>

In [25]:
booked_counts = is_booked["duration"].value_counts().sort_index()
not_booked_counts = not_booked["duration"].value_counts().sort_index()

total_booked = booked_counts.sum()
total_not_booked = not_booked_counts.sum()
booked_percent = (booked_counts / total_booked ) if total_booked > 0 else pd.Series(0, index=hours)
not_booked_percent = (not_booked_counts / total_not_booked ) if total_not_booked > 0 else pd.Series(0, index=hours)
trace_not_booked = go.Scatter(y = not_booked_percent , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y =booked_percent  , name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Length of Stay', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./los.json')

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خارجی یا داخلی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
تا اینجا داده‌های زمانی را مورد بررسی قرار دادیم و سعی کردیم کمی در آن‌‌ها اکتشاف کنیم.
حال با توجه به داده‌های دیگر می‌توانیم اکتشافات دیگری انجام دهیم. <br>ستونی با نام <code>is_abroad</code> ایجاد کنید و در آن جست‌وجو‌ها برای هتل‌های خارج از کشور کاربر را مشخص کنید.
به‌طور مثال اگر کاربر در کشوری با شناسه‌ی <code>x</code> است و در جست‌وجوی خود هتلی در کشوری غیر از <code>x</code> را جست‌وجو ستون <code>is_abroad</code> را برای این نمونه برابر <code>1</code> قرار دهید.
</font>
</p>


In [26]:
train['is_abroad'] = (train['user_location_country'] != train['hotel_country']).astype(int)
test['is_abroad'] = (test['user_location_country'] != test['hotel_country']).astype(int)

In [29]:
train.head()

user_location_country  user_location_region  user_location_city  \
0                      3                    50                5703   
1                     66                   174               21177   
2                    205                   155               14703   
3                     69                   761               41949   
4                     69                   761               41949   

   destination_distance         search_date  is_mobile  is_package  channel  \
0              0.000000 2013-01-07 00:00:02          0           0        9   
1           5713.620600 2013-01-07 00:00:06          0           0        9   
2            795.729800 2013-01-07 00:00:06          0           0        9   
3            402.699108 2013-01-07 00:00:17          0           1        9   
4            402.699108 2013-01-07 00:00:23          0           1        9   

   search_count checkIn_date  ... search_date_month  checkIn_date_month  \
0             1   2013-03-14  ...                 1                   3   
1             3   2013-01-19  ...                 1                   1   
2             1   2013-01-19  ...                 1                   1   
3             1   2013-05-29  ...                 1                   5   
4             1   2013-05-29  ...                 1                   5   

   search_hour_bin  n_adults_mapped  n_child_mapped  total_persons  \
0                0                2               1              3   
1                0                1               0              1   
2                0                1               0              1   
3                0                2               0              2   
4                0                2               0              2   

   persons_per_room  is_room_sufficient  group_type  is_abroad  
0          2.999997                   1           2          1  
1          0.999999                   1           0          1  
2          0.999999                   1           0          1  
3          1.999998                   1           1          1  
4          1.999998                   1           1          1  

[5 rows x 36 columns]

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
🤓 بیشتر
</font>
</h3>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
از آن‌جا که این مجموعه‌داده شامل ویژگی‌های متعدد و متنوعی است، می‌توانید به غیر از موارد خواسته‌شده، از جنبه‌های مختلفی به تحلیل داده‌ها بپردازید. برای مثال می‌توانید توزیع‌های سایر ویژگی‌ها را بررسی کنید، ارتباط ویژگی‌ها با هم و با متغیر هدف را بررسی کنید. همچنین با توجه به بینش‌هایی که از داده‌ها به‌دست می‌آورید، می‌توانید به پیش‌پردازش داده‌ها بپردازید تا ورودی‌های باکیفیت‌تری را برای مدل خود آماده کنید. این بخش به‌صورت جداگانه مورد داوری قرار نمی‌گیرد و دست شما باز است تا از هر تکنیکی که برای بهبود عملکرد مدل خود لازم می‌دانید استفاده کنید.
در انجام مراحل پیش‌پردازش و مهندسی ویژگی، می‌توانید موارد زیر را انجام دهید. همچنین توجه داشته باشید در نهایت مجموعه‌داده آموزش و آزمون باید ساختاری مشابه یکدیگر داشته باشند.

<ul dir=rtl>
<li dir=rtl>
 مدیریت ویژگی‌های دسته‌ای ترتیبی و اسمی (توجه داشته باشید با این‌که داده‌های مجموعه‌داده‌ی اولیه از نوع عدد هستند اما الزاماً اعداد آن‌ها بار معنایی از نظر ریاضی ندارند و با برخی از آن‌ها باید همچون ویژگی‌های دسته‌ای برخورد کنید و شاید برای بهبود مدل نیاز به مدیریت این داده‌ها داشته باشید.)
    </li>
    <li dir=rtl>
        در بعضی موارد حذف بعضی ستون‌ها به بهبود مدل کمک می‌کند، به‌طور مثال در مرحله‌ی قبل ستونی برای خارجی بودن هتل ایجاد کردیم، حال ممکن است دیگر نیازی به کشور کاربر و یا هتل با توجه به ستون جدید نداشته باشیم.
    </li>
       <li dir=rtl>
           با بررسی توزیع مجموعه‌داده متوجه می‌شوید که این مجموعه‌داده متوازن نیست، بنابراین نحوه‌ی مدیریت این چالش نیز می‌تواند در مدل شما تاثیر بگذارد.
    </li>
</ul>
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
توجه داشته باشید که کدنویسی سلول‌های زیر کاملاً اختیاری است.
</font>
</p>

In [27]:
train = train.drop(['search_date', 'checkIn_date', 'checkOut_date', 'n_adults', 'n_children'], axis=1)
test = test.drop(['search_date', 'checkIn_date', 'checkOut_date', 'n_adults', 'n_children'], axis=1)

In [32]:
# بررسی واریانس کم
selector = VarianceThreshold(threshold=0.01)
selector.fit(train.drop('is_booking', axis=1))
low_variance_cols = train.drop('is_booking', axis=1).columns[~selector.get_support()]
print("ستون‌های با واریانس کم:", low_variance_cols)
train = train.drop(low_variance_cols, axis=1)
test = test.drop(low_variance_cols, axis=1)

ستون‌های با واریانس کم: Index(['is_abroad'], dtype='object')


In [24]:
numerical_cols = ['destination_distance', 'persons_per_room', 'duration', 'days_between', 'search_count', 'total_persons']
categorical_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'channel', 'destination', 
                    'destination_type', 'hotel_continent', 'hotel_country', 'hotel_market', 'group_type']

In [35]:
# بررسی هم‌خطی
corr_matrix = train[numerical_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print("ستون‌های با هم‌خطی بالا:", to_drop)
train = train.drop(to_drop, axis=1)
test = test.drop(to_drop, axis=1)

ستون‌های با هم‌خطی بالا: []


In [28]:
# مهندسی ویژگی‌های جدید
train['is_long_stay'] = (train['duration'] > 7).astype(int)
test['is_long_stay'] = (test['duration'] > 7).astype(int)
train['is_early_booking'] = (train['days_between'] > 30).astype(int)
test['is_early_booking'] = (test['days_between'] > 30).astype(int)
train['is_weekend_checkin'] = (train['checkIn_date_dayofweek'] >= 5).astype(int)
test['is_weekend_checkin'] = (test['checkIn_date_dayofweek'] >= 5).astype(int)


In [27]:
train.dtypes

user_location_country       int64
user_location_region        int64
user_location_city          int64
destination_distance      float64
is_mobile                   int64
is_package                  int64
channel                     int64
search_count                int64
n_rooms                     int64
destination                 int64
destination_type            int64
hotel_continent             int64
hotel_country               int64
hotel_market                int64
hotel_category              int64
is_booking                  int64
duration                    int64
days_between                int64
search_date_hour            int32
search_date_dayofweek       int32
checkIn_date_dayofweek      int32
search_date_month           int32
checkIn_date_month          int32
search_hour_bin             int64
n_adults_mapped             int64
n_child_mapped              int64
total_persons               int64
persons_per_room          float64
is_room_sufficient          int32
group_type    

In [29]:
train.shape

(7608708, 34)

In [30]:
# نرمال‌سازی ویژگی‌های عددی
numerical_cols = ['destination_distance', 'duration', 'days_between', 'search_count', 'total_persons','n_rooms','n_adults_mapped','n_child_mapped','persons_per_room','search_date_hour','search_date_dayofweek','checkIn_date_dayofweek','search_date_month','checkIn_date_month']
scaler = StandardScaler()
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])
test[numerical_cols] = scaler.transform(test[numerical_cols])

In [ ]:
categorical_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'channel', 'destination', 
                    'destination_type', 'hotel_continent', 'hotel_country', 'hotel_market', 'group_type','n_adults_mapped','n_child_mapped',]
for col in categorical_cols:
    if col in train.columns:
        print(f"{col}: {train[col].nunique()}")

user_location_country: 236
user_location_region: 983
user_location_city: 41174
channel: 11
destination: 42833
destination_type: 9
hotel_continent: 7
hotel_country: 211
hotel_market: 2115
group_type: 5
n_adults_mapped: 3
n_child_mapped: 4


In [32]:
train = train.drop(['user_location_city', 'destination', 'hotel_market', 'user_location_region'], axis=1)
test = test.drop(['user_location_city', 'destination', 'hotel_market', 'user_location_region'], axis=1)

In [35]:
categorical_cols = [ 'channel', 'user_location_country'
                    ,'destination_type', 'hotel_continent', 'hotel_country',  'group_type','n_adults_mapped','n_child_mapped']
binary_encoder = ce.BinaryEncoder(cols=categorical_cols)
train_binary = binary_encoder.fit_transform(train[categorical_cols])
test_binary = binary_encoder.transform(test[categorical_cols])
train = train.drop(categorical_cols, axis=1)
test = test.drop(categorical_cols, axis=1)
train = pd.concat([train, train_binary], axis=1)
test = pd.concat([test, test_binary], axis=1)

C:\Users\MSI\AppData\Roaming\Python\Python310\site-packages\category_encoders\ordinal.py:198: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\MSI\AppData\Roaming\Python\Python310\site-packages\category_encoders\ordinal.py:198: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\MSI\AppData\Roaming\Python\Python310\site-packages\category_encoders\ordinal.py:198: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to 

In [36]:
train.shape

(7608708, 57)

In [37]:
X_test = test.drop('Unnamed: 0', axis=1)


In [38]:
# آماده‌سازی داده‌ها
X = train.drop('is_booking', axis=1)
y = train['is_booking']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
 


In [28]:
train.dtypes

user_location_country              int64
user_location_region               int64
user_location_city                 int64
destination_distance             float64
search_date               datetime64[ns]
is_mobile                          int64
is_package                         int64
channel                            int64
search_count                       int64
checkIn_date              datetime64[ns]
checkOut_date             datetime64[ns]
n_adults                           int64
n_children                         int64
n_rooms                            int64
destination                        int64
destination_type                   int64
hotel_continent                    int64
hotel_country                      int64
hotel_market                       int64
hotel_category                     int64
is_booking                         int64
duration                           int64
days_between                       int64
search_date_hour                   int32
search_date_dayo

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از آماده‌سازی داده‌ها نوبت به ساخت شبکه‌ی عصبی می‌رسد. باز هم دست‌تان باز است تا معماری موردنظر خود را طراحی کنید و از هر تکنیکی که لازم می‌دانید جهت بهتر کردن فرآیند آموزش مدل استفاده کنید. پیشنهاد می‌کنیم تکنیک‌های مختلفی که تا اینجای دوره آموخته‌اید را آزمایش کرده و به تفسیر نتایج بپردازید.
دقت داشته باشید در خروجی مدل باید احتمال رزرو شدن هتل را پیش‌بینی کنید. به عبارت بهتر احتمال <code>1</code>
 شدن ستون <code>is_booking</code>
 را تخمین بزنید.
همچنین توجه داشته باشید که برای ارزیابی مدل شما از معیار <code>AUC</code> یعنی سطح زیر منحنی نمودار <code>ROC</code> استفاده می‌شود. برای مطالعه بیشتر در مورد این نمودار می‌توانید <a href = 'https://en.wikipedia.org/wiki/Receiver_operating_characteristic'> ویکی‌پدیا </a>
    یا <a href = 'https://stanford.edu/~shervine/l/fa/teaching/cs-229/cheatsheet-machine-learning-tips-and-tricks'> راهنمای کوتاه نکات و ترفندهای یادگیری ماشین </a>
 را مطالعه کنید.
</font>
</p>

In [42]:
class_weight = {0: 1.0, 1: len(y_train[y_train==0])/len(y_train[y_train==1])}

In [ ]:
# تعریف مدل دیپ لرنینگ پیشرفته‌تر
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.005)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# کامپایل مدل
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# آموزش مدل
history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=5,
                    validation_data=(X_valid, y_valid),class_weight=class_weight,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
                    verbose=1)







# پیش‌بینی روی داده تست

Epoch 1/5
23778/23778 ━━━━━━━━━━━━━━━━━━━━ 195s 8ms/step - accuracy: 0.5729 - loss: 1.2636 - val_accuracy: 0.5806 - val_loss: 0.5951
Epoch 2/5
23778/23778 ━━━━━━━━━━━━━━━━━━━━ 204s 9ms/step - accuracy: 0.5998 - loss: 1.0439 - val_accuracy: 0.5916 - val_loss: 0.6020
Epoch 3/5
23778/23778 ━━━━━━━━━━━━━━━━━━━━ 176s 7ms/step - accuracy: 0.6034 - loss: 1.0373 - val_accuracy: 0.6458 - val_loss: 0.5518
Epoch 4/5
23774/23778 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6089 - loss: 1.0339

In [ ]:
# ارزیابی مدل
y_pred_proba = model.predict(X_valid, batch_size=512)  # احتمالات
y_pred = (y_pred_proba >= 0.5).astype(int)  # پیش‌بینی‌های باینری
accuracy = (y_pred.flatten() == y_valid).mean()
f1 = f1_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred)
recall = recall_score(y_valid, y_pred)
roc_auc = roc_auc_score(y_valid, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

In [41]:
from sklearn.metrics import roc_auc_score
print("roc_auc_score:", roc_auc_score(y_valid, y_pred))


roc_auc_score: 0.5


In [ ]:
# پیش‌بینی روی داده تست
predictions = (model.predict(X_test)).astype(int)
submission = pd.DataFrame(predictions, columns=["is_booking"])

In [ ]:
# آماده‌سازی داده‌ها
X = train.drop('is_booking', axis=1)
y = train['is_booking']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = test


# تعریف مدل دیپ لرنینگ
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# کامپایل مدل
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# آموزش مدل
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=1,  
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
                    verbose=1)



In [40]:
train.dtypes

user_location_country       int64
user_location_region        int64
user_location_city          int64
destination_distance      float64
is_mobile                   int64
is_package                  int64
channel                     int64
search_count                int64
n_adults                    int64
n_children                  int64
n_rooms                     int64
destination                 int64
destination_type            int64
hotel_continent             int64
hotel_country               int64
hotel_market                int64
hotel_category              int64
is_booking                  int64
duration                    int64
days_between                int64
search_date_hour            int32
search_date_dayofweek       int32
checkIn_date_dayofweek      int32
search_date_month           int32
checkIn_date_month          int32
search_hour_bin             int64
n_adults_mapped             int64
n_child_mapped              int64
total_persons               int64
persons_per_ro

In [ ]:
test.hea

Unnamed: 0  user_location_country  user_location_region  \
0           0                    215                   817   
1           1                     66                   321   
2           2                    109                     0   
3           3                     66                   174   
4           4                    215                   646   

   user_location_city  destination_distance  is_mobile  is_package  channel  \
0               19599                0.0000          0           0        9   
1               47535             7322.6527          0           0        0   
2               21903                0.0000          0           1        9   
3               50284                0.0000          0           0        9   
4               51733             2045.2943          0           0        9   

   search_count  n_adults  ...  n_child_mapped  total_persons  \
0             1         4  ...               0              4   
1             1         1  ...               0              1   
2             1         1  ...               0              1   
3             1         2  ...               0              2   
4             1         2  ...               0              2   

   persons_per_room  is_room_sufficient  group_type  is_abroad  is_long_stay  \
0          3.999996                   1           4          1             0   
1          0.999999                   1           0          1             0   
2          0.999999                   1           0          1             0   
3          1.000000                   1           3          1             0   
4          1.999998                   1           1          1             0   

   is_early_booking  is_weekend_checkin  n_adults_per_room  
0                 0                   1           2.999997  
1                 0                   0           0.999999  
2                 0                   0           0.999999  
3                 0                   0           1.000000  
4                 0                   0           1.999998  

[5 rows x 37 columns]

In [46]:
missing_cols =  set(test.columns) -set(train.columns) 
print(missing_cols)

{'Unnamed: 0'}


In [51]:
pd.DataFrame(y_valid.copy()).head(20)

0
0   0
1   0
2   0
3   0
4   0
5   0
6   1
7   0
8   0
9   1
10  0
11  0
12  0
13  0
14  0
15  0
16  0
17  0
18  0
19  0

In [53]:
from sklearn.metrics import roc_auc_score
y_pred = (model.predict(X_valid) >= 0.5).astype(int)
print("roc_auc_score:", roc_auc_score(y_valid, y_pred))

# پیش‌بینی روی داده تست
predictions = (model.predict(X_test)).astype(int)
submission = pd.DataFrame(predictions, columns=["is_booking"])

47555/47555 ━━━━━━━━━━━━━━━━━━━━ 85s 2ms/step
roc_auc_score: 0.5
10182/10182 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step


In [29]:
test=test.drop(["search_date","checkIn_date","checkOut_date"],axis=1)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در فایل <code>test.csv</code> لیستی از نمونه‌های آزمون در اختیار شما قرار گرفته و نیاز است به‌ازای هر کدام از آن‌ها احتمال رزرو هتل را پیش‌بینی کنید. پیشنهاد می‌کنیم برای اطمینان از عملکرد مدل خود و بهبود آن، ابتدا آن را بر روی یک مجموعه‌ی اعتبارسنجی آزمایش کنید.
</font>
</p>


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
برای ارزیابی عملکرد مدل شما بر روی نمونه‌های آزمون از معیار <code>AUC</code> نمودار <code>ROC</code> استفاده می‌کنیم. 
</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>74.00</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

In [ ]:
from sklearn.metrics import roc_auc_score

# TODO: Evaluate Model

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساختار خروجی برای نمونه‌های آزمون
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون نیاز است به‌ازای هر نمونه‌ی آزمون <b style="color:#0099cc">احتمال</b> رزرو هتل را پیش‌بینی کنید.
 پیش‌بینی مدل خود را برای نمونه‌های آزمون تولید کرده و در ستون <code>prediction</code> دیتافریمی با نام <code>submission</code> ذخیره کنید. توجه داشته باشید که ترتیب مقادیر این ستون باید دقیقاً معادل با ترتیب نمونه‌های آزمون باشد. به این معنی که  ردیف i ام ستون <code>submission</code>، پیش‌بینی شما برای سطر ردیف <code>i</code> ام فایل آزمون باشد (دقت کنید که این ستون باید حتماً دارای <code>header</code> باشد).  همچنین از آنجایی که این ستون بیانگر احتمال است مقادیر این ستون باید مقداری <b>بین</b> <code>0</code> تا <code>1</code> باشد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
یک نمونه‌ی فرضی از دیتافریم <code>submission</code> در جدول زیر نمایش داده شده است.
 </font>
</p>
<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|<code>prediction</code>|
|:------:|
|0.723|
|0.516|
|0.281|
| ... |

    
</font>
</div>
</center>

In [ ]:
# Preparing Submission Dataframe

submission = None # TODO

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد. همچنین اگر از گوگل کولب استفاده می‌کنید، در صورت نیاز به پشتیبانی حتماً آخرین نسخه از نت‌بوک را به‌صورت دستی دانلود کرده و داخل فایل ارسالی قرار دهید یا لینک کولب را با ما به‌اشتراک بگذارید.
</font>
</p>

In [ ]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'will_not_travel_again.ipynb')):
    %notebook -e will_not_travel_again.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['will_not_travel_again.ipynb', 'submission.csv',
              'search_hour.json', 'checkIn_date_month.json',
              'checkIn_day.json', 'days_between.json', 'los.json']
compress(file_names)